<a href="https://colab.research.google.com/github/EyalKoubi/CloudProject/blob/main/HW3_Dolev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install firebase

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
################ imports #########################################################################
import ipywidgets as widgets
from IPython.display import display,HTML
import random
from firebase import firebase
import time
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.widgets import Button
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from IPython.display import clear_output
from IPython.display import display
import io
import IPython.display
from matplotlib.figure import Figure

# Initialize the Firebase connection
FBconn = firebase.FirebaseApplication('https://new-proj-917e8-default-rtdb.firebaseio.com/', None)

############################### get function #######################################################

# Function to retrieve all questions from Firebase
def get_all_questions():
    result = FBconn.get('/Questions', None)
    questions = []
    if result:
        for key, value in result.items():
            options = [value['Option1'],value['Option2'],value['Option3'],value['Option4']]
            question = {
                'question': value['Question'],
                'options': options,
                'correctOption': value['Correct']
            }
            questions.append(question)
    return questions

####################################### statistic screen ######################################################

def statisticScreen(right_answers, response_times, total_time):
    # Calculate the average response time for right answers
    right_response_times = [t for t in response_times if t >= 0]
    avg_response_time = np.mean(right_response_times)

    # Generate the data for the bar chart
    categories = list(range(1, len(response_times) + 1))
    response_times = [t if t >= 0 else np.nan for t in response_times]

    # Create the figure and axes for the graph
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.bar(categories, response_times, color='lightblue', alpha=0.8)

    # Set axes labels and title
    ax.set_xlabel('Question')
    ax.set_ylabel('Response Time (seconds)')
    ax.set_title('Quiz Game Results')

    # Customize the graph appearance
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.7)

    # Save the current figure to a BytesIO object
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    # Create an IPython Image from this object
    plot_img = IPython.display.Image(buf.read(), format='png')
    buf.close()

    # Create a widgets.Image, ready to be displayed
    plot_widget = widgets.Image(value=plot_img.data, format='png')

    # Create the text annotations for details
    points_text = f'Correct Answers: {right_answers}'
    wrong_answers_text = f'Wrong Answers: {10-right_answers}'
    total_time_text = f'Total Time: {total_time} seconds'
    if right_response_times != []:
        fastest_time_text = f'Fastest Time: {min(right_response_times)} seconds'
    else:
        fastest_time_text = f'Fastest time: None'
    avg_time_text = f'Average Time: {avg_response_time:.2f} seconds'
    details_widget = widgets.HTML(f"<div style='display: inline-block; border-style: dotted; border-color: RoyalBlue; background-color: lightblue; color: black; margin: 15px; padding:25px; font-size:24px;'> {points_text}&nbsp;&nbsp;&nbsp;{wrong_answers_text}&nbsp;&nbsp;&nbsp;{total_time_text}<br>{fastest_time_text}&nbsp;&nbsp;&nbsp;{avg_time_text} </div>")
    details_widget.add_class("details")

    # Display the image and details in a vertical box
    graph_details_container = widgets.VBox([plot_widget, details_widget], layout=widgets.Layout(height='auto', justify_content='center', align_items='center'))

    # Close the figure
    plt.close(fig)

    # Create the Back and Try Again buttons
    btn_back = widgets.Button(description='Back', layout=widgets.Layout(width='250px', height='auto'))
    btn_back.add_class("button")
    btn_try_again = widgets.Button(description='Try Again', layout=widgets.Layout(width='250px', height='auto'))
    btn_try_again.add_class("button")

    # Function to handle button clicks
    def handle_button_click(b):
        if b.description == 'Back':
            clear_output(wait=True)
            display(createPlayerScreen())
        elif b.description == 'Try Again':
            clear_output(wait=True)
            display(createSelectionScreen())

    # Assign the button click handlers
    btn_back.on_click(handle_button_click)
    btn_try_again.on_click(handle_button_click)

    # Create a container for the buttons
    buttons_container = widgets.HBox([btn_back, btn_try_again])

    # main_container.children = [container,buttons_container]
    main_container = widgets.VBox([graph_details_container, buttons_container], layout=widgets.Layout(width='100%', height='auto', justify_content='center', align_items='center'))
    main_container.add_class("background-image")

    # Apply CSS styles
    css = """
    <style>
      .question {
        color: #4169E1;
        text-align: center;
        font-size: 30px;
        padding: 50px 50px;
        text-transform: uppercase;
    }

    .details {
        align: center;
    }

    .button {
      display: inline-block;
      padding: 15px 25px;
      font-size: 24px;
      cursor: pointer;
      text-align: center;
      text-decoration: none;
      outline: none;
      color: #F0F8FF;
      background-color: #6495ED;
      border: none;
      border-radius: 15px;
      box-shadow: 0 9px #4682B4;
      margin: 25px 50px 75px 100px
    }

    .button:hover {background-color: #F0F8FF; color: #6495ED;  margin: 25px 50px 75px 100px}

    .button:active {
      background-color: #4169E1;
      color: #F0F8FF;
      box-shadow: 0 5px #B0E0E6;
      margin: 25px 50px 75px 100px;
      transform: translateY(4px);
    }

    .background-image {
        background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
        background-repeat: no-repeat;
        background-position: center;
        background-size: cover;
    }
    </style>
    """

    # Return the main_container
    return main_container

    

####################################### quiz screen ######################################################

def quizScreen(topics):
    # Create buttons
    back_button = widgets.Button(description='Back', layout=widgets.Layout(width='auto', height='auto'))
    next_button = widgets.Button(description='Next', layout=widgets.Layout(width='auto', height='auto'))
    next_button.add_class("button")
    back_button.add_class("button")

    answer_chosen = False

    # Create a question widget
    question_widget = widgets.HTML()
    question_widget.add_class("question")

    # Create a box to hold the 4 options buttons
    options_container = widgets.VBox()

    # Create a box to hold the question and options
    question_container = widgets.VBox([question_widget, options_container ], layout=widgets.Layout(justify_content='center', align_items='center'))

    # Create a list to store burndown icons
    burndown_icons = []

    # Create a burndown graph widget
    burndown_graph = widgets.HTML()
    burndown_graph.layout.margin = '10px'

    # Get all questions
    questions = get_all_questions()

    # Shuffle the questions randomly
    random.shuffle(questions)

    picked = 10 if len(questions) >= 10 else len(questions)

    temp = []

    for i in range(picked):
      temp.append(questions[i])

    questions = temp

    # Set the initial question index to 0
    current_question_index = 0

    # Create an array to store the time taken for each question
    time_array = [-1] * len(questions)

    # Create a variable to store the total time taken
    total_time = 0

    # Function to update the question and options
    def update_question():
        nonlocal answer_chosen
        nonlocal current_question_index
        nonlocal total_time
        answer_chosen = False
        
        # Start the timer for the current question
        start_time = time.time()

        # Check if all questions have been answered
        if current_question_index >= len(questions):
            time.sleep(1)
            num_correct = 0  # Variable to store the number of correct answers

            for i in range(len(burndown_icons)):
                total_time += time_array[i]
                if burndown_icons[i] == 'X':
                    time_array[i] = -1
                else:
                    num_correct += 1
            total_time = round(total_time, 2)
            screen_container.close()
            display(statisticScreen(num_correct, time_array, total_time))
            return

        # Get the current question and options
        question = questions[current_question_index]['question']
        options = questions[current_question_index]['options']

        # Update the question widget and options widget
        question_widget.value = f"<p>{question}<p>"
        # Create 4 button widgets for options
        option1_button = widgets.Button(description=options[0])
        option2_button = widgets.Button(description=options[1])
        option3_button = widgets.Button(description=options[2])
        option4_button = widgets.Button(description=options[3])
        options_container.children = [option1_button, option2_button, option3_button, option4_button]
        option1_button.add_class("option-button")
        option2_button.add_class("option-button")
        option3_button.add_class("option-button")
        option4_button.add_class("option-button")
        

        # Function to enable or disable the option buttons and next button
        def buttons_state(state):
          option1_button.disabled = state
          
          option2_button.disabled = state
          
          option3_button.disabled = state
          
          option4_button.disabled = state
          
          next_button.disabled = not state
          if state:
              option1_button.add_class("disabled-cursor")
              option2_button.add_class("disabled-cursor")
              option3_button.add_class("disabled-cursor")
              option4_button.add_class("disabled-cursor")
              next_button.remove_class("disabled-cursor")
          else:
              option1_button.remove_class("disabled-cursor")
              option2_button.remove_class("disabled-cursor")
              option3_button.remove_class("disabled-cursor")
              option4_button.remove_class("disabled-cursor")   
              next_button.add_class("disabled-cursor")
       
              
        buttons_state(False)  

        # Define a common function for button clicks
        def option_button_clicked(button):
          nonlocal answer_chosen
          nonlocal burndown_icons
          update_time()
          # Check if an answer is already chosen
          if answer_chosen:
                return

          buttons_state(True)
          # Mark that an answer is chosen
          answer_chosen = True      
          # Check if the selected option is correct
          selected_option = button.description
          correct_option = questions[current_question_index]['correctOption']
          is_correct = selected_option == correct_option

          # Change button colors based on correctness
          if is_correct:
            button.style.button_color = 'lightgreen'
            # Add 'V'  to burndown_icons
            burndown_icons.append('V')
          else:
            button.style.button_color = 'red'
            correct_button = get_correct_option_button(correct_option)
            correct_button.style.button_color = 'lightgreen'
            # Add 'X'  to burndown_icons
            burndown_icons.append('X')

            
        # Function to get the button corresponding to the correct option
        def get_correct_option_button(correct_option):
          if correct_option == options[0]:
             return option1_button
          elif correct_option == options[1]:
            return option2_button
          elif correct_option == options[2]:
            return option3_button
          else: 
            return option4_button

        # Assign the common function to the on_click event of each button
        option1_button.on_click(option_button_clicked)
        option2_button.on_click(option_button_clicked)
        option3_button.on_click(option_button_clicked)
        option4_button.on_click(option_button_clicked)

        
        # Function to calculate and update the time taken for the current question
        def update_time():
            nonlocal current_question_index

            if current_question_index < len(questions):
                current_time = time.time() - start_time
                time_array[current_question_index] = round(current_time, 2)


       

    # Function to handle button click event
    def next_button_clicked(button):
        nonlocal current_question_index
        
        # Increment the current_question_index
        if current_question_index<len(questions):
          current_question_index += 1

        # Update the burndown graph
        update_burndown_graph()

        # Update the question and options
        update_question()

    # Function to update the burndown graph
    def update_burndown_graph():
        graph_html = ''
        for icon in burndown_icons:
            if icon=='V':
              graph_html += '<span style="margin-right:7px;"><img src="https://github.com/EyalKoubi/CloudProject/blob/main/rightAnswer.png?raw=true" align_items="center", width=50px, height=50px></span>'
            elif icon == 'X':
              graph_html += '<span style="margin-right:7px;"><img src="https://github.com/EyalKoubi/CloudProject/blob/main/wrongAnswer.png?raw=true" align_items="center", width=50px, height=50px></span>'
        graph_html += ''
        burndown_graph.value = graph_html
    def back_button_clicked(button):
        screen_container.close()
        display(topicScreen())

    back_button.on_click(back_button_clicked)
    # Update the question and options initially
    update_question()

    # Create a box to hold the burndown graph, next and back button
    buttons_container = widgets.HBox([back_button, next_button], layout=widgets.Layout(justify_content='space-between' , width = '800px'))

    # Create a box to hold the entire screen
    screen_container = widgets.VBox([burndown_graph, question_container, buttons_container],layout=widgets.Layout(width='100%', height='700px', justify_content='center', align_items='center'))
    screen_container.add_class("background-image")

    # Display the screen container
    display(screen_container)

    # Register the button click event handler
    next_button.on_click(next_button_clicked)

    # Apply CSS styles
    css = """
    <style>
      .question {
        color: #00079C;
        text-align: center;
        font-size: 20px;
        padding: 50px 50px;
        text-transform: uppercase;
      }

      .button {
        display: inline-block;
        padding: 15px 25px;
        font-size: 24px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 15px;
        box-shadow: 0 9px #4682B4;
        margin: 25px 50px 75px 
      }

      .button:hover {
        background-color: #F0F8FF;
        color: #6495ED; 
        margin: 25px 50px 75px 
        }

      .button:active {
        background-color: #4169E1;
        color: #F0F8FF;
        box-shadow: 0 5px #B0E0E6;
        margin: 25px 50px 75px 100px;
        transform: translateY(4px);
      }
     .option-button {
        display: inline-block;
        padding: 10px 30px;
        font-size: 13px;
        cursor: pointer;
        text-align: center;
        text-decoration: none;
        outline: none;
        color: #F0F8FF;
        background-color: #6495ED;
        border: none;
        border-radius: 10px;
        box-shadow: 0 5px #4682B4;
        margin: 10px 0;
        width: 700px;
        line-height: 13px;
      }
      .disabled-cursor {
        cursor: not-allowed !important;
      }
    
      .background-image {
          background-image: url('https://w.forfun.com/fetch/3d/3dd0b8f4ab9d0752d3ca9a569723efb3.jpeg?w=1470&r=0.5625');
          background-repeat: no-repeat;
          background-position: center;
          background-size: cover;
      }
    </style>
    """
    display(HTML(css))
quizScreen('a')
